In [49]:
%serialconnect

Connecting to --port=/dev/ttyUSB2 --baud=115200 
Ready.


This is the main Sensor Kennel application that will 
identify, read and write any sensor that's plugged in.

On the lower side:  
  Pin1 (left) Gnd, Pin3 3v3, Pin-3 (from right) SDA, Pin-5 SCL.
  
Do we want to draw a graph of numbers if a button is pressed?

Should we have MQTT numbers going out from it?

Or just keep it comprehensive

Sensors known:
* VL53L0X lidar
* VL6180 lidar
* SI7021 humidity
* TSL561 luminous
* TMP102 temp
* SDOF GyAccMag
* BME280 barhumid
* SHT31D tmphumid
* MLX90621 16x4-ir
* PX4PITOT airspeed
* BNO055 orient
* GPS serial


In [6]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [7]:
%sendtofile --binary --source OLED_driver.py
%sendtofile --quiet --source device_detection.py
%sendtofile --quiet --source SI7021_funcs.py
%sendtofile --quiet --source TSL561_funcs.py
%sendtofile --quiet --source VL53L0X_funcs.py
%sendtofile --quiet --source VL6180_funcs.py
%sendtofile --quiet --source SDOF_funcs.py
%sendtofile --quiet --source SHT31D_funcs.py
%sendtofile --quiet --source BME280_funcs.py
%sendtofile --quiet --source MLX90621_funcs.py
%sendtofile --quiet --source BNO055serial_funcs.py


Sent 3170 bytes in 106 chunks to OLED_driver.py.
Sent 91 lines (2742 bytes) to device_detection.py.
Sent 55 lines (2042 bytes) to SI7021_funcs.py.
Sent 54 lines (1757 bytes) to TSL561_funcs.py.
Sent 33 lines (975 bytes) to VL53L0X_funcs.py.
Sent 106 lines (3190 bytes) to VL6180_funcs.py.
Sent 42 lines (1800 bytes) to SDOF_funcs.py.
Sent 43 lines (1090 bytes) to SHT31D_funcs.py.
Sent 75 lines (3137 bytes) to BME280_funcs.py.
Sent 166 lines (5743 bytes) to MLX90621_funcs.py.
Sent 88 lines (2732 bytes) to BNO055serial_funcs.py.


In [39]:
%sendtofile main.py

# connect to the OLED and initialize (and get its i2c bus)
from OLED_driver import i2c, fbuff, oledshow, doublepixels, fatntext
import machine, time
time.sleep(0.5)

from device_detection import IdentifyI2CDevice, IdentifyUARTDevice

devices = IdentifyI2CDevice(i2c)
if not devices:
    uart = machine.UART(1, baudrate=9600, rx=13, tx=17)
    devices = IdentifyUARTDevice(uart)
if not devices:
    devices.append("nothing found")

# this might look like a lot of code, but it's the complete 
# visual output of text and numbers that we do per sensor here
for dev in devices:
    devname, devdesc = dev.split(" ", 1)
    fbuff.fill(0)
    fbuff.text(devname, 0, 0, 1)
    fbuff.text(devdesc, 0, 12, 1)
    doublepixels()
    oledshow()
    time.sleep(2)
    
if devname == "SI7021":
    from SI7021_funcs import setupSI7021, SI7021humiditytemp
    setupSI7021(i2c)
    while True:
        h, t = SI7021humiditytemp()
        fbuff.fill(0)
        fbuff.text("SI7021", 0, 0, 1)
        fbuff.text("humid:", 0, 23, 1)
        fatntext("%.1f"%h, 50, 16)
        fbuff.text("temp:", 0, 53, 1)
        fatntext("%.1f"%t, 50, 46)
        oledshow()

if devname == "VL53L0X":
    from VL53L0X_funcs import VL53L0Xinit, VL53L0Xdist
    VL53L0Xinit(i2c)
    while True:
        d = VL53L0Xdist()
        fbuff.fill(0)
        fbuff.text("VL53L0X", 0, 0, 1)
        fbuff.text("dist(mm):", 8, 20, 1)
        fatntext("%d" % d, 40, 36)
        oledshow()
        
if devname == "VL6180":
    from VL6180_funcs import VL6180init, distmm
    VL6180init(i2c)
    while True:
        d = distmm()
        fbuff.fill(0)
        fbuff.text("VL53L0X", 0, 0, 1)
        fbuff.text("dist(mm):", 8, 20, 1)
        fatntext("%d" % d, 40, 36)
        oledshow()

if devname == "TSL561":
    from TSL561_funcs import setupTSL561, luminosity, setregtimings
    setupTSL561(i2c)
    setregtimings(True, 1)
    while True:
        br, ir = luminosity()
        fbuff.fill(0)
        fbuff.text("TSL561", 0, 0, 1)
        fbuff.text("broad:", 0, 23, 1)
        fatntext("%d"%br, 50, 16)
        fbuff.text("ir:", 0, 53, 1)
        fatntext("%d"%ir, 50, 46)
        oledshow()
        time.sleep(0.1)

if devname == "TMP102":
    while True:
        r = i2c.readfrom_mem(0x48, 0x00, 2)
        t = r[0] + r[1]/256
        fbuff.fill(0)
        fbuff.text("TMP102", 0, 0, 1)
        fbuff.text("temp:", 0, 23, 1)
        fatntext("%.03f"%t, 30, 40)
        oledshow()
        time.sleep(0.1)

if devname == "SDOF":
    from SDOF_funcs import SetupAccGyrMag, readvectorsensor
    SetupAccGyrMag(i2c)
    while True:
        fbuff.fill(0)
        x, y, z = readvectorsensor("a")
        fbuff.text("Acc:", 0, 0, 1)
        fbuff.text("%d"%x, 8, 8, 1)
        fbuff.text("%d"%y, 8, 16, 1)
        fbuff.text("%d"%z, 8, 24, 1)
        x, y, z = readvectorsensor("g")
        fbuff.text("gyr:", 64, 0, 1)
        fbuff.text("%d"%x, 72, 8, 1)
        fbuff.text("%d"%y, 72, 16, 1)
        fbuff.text("%d"%z, 72, 24, 1)
        x, y, z = readvectorsensor("c")
        fbuff.text("mag:", 40, 32, 1)
        fbuff.text("%d"%x, 48, 40, 1)
        fbuff.text("%d"%y, 48, 48, 1)
        fbuff.text("%d"%z, 48, 56, 1)
        oledshow()
        time.sleep(0.1)
        
if devname == "BME280" or devname == "BME180":
    from BME280_funcs import bme280init, readBME280
    bme280init(i2c)
    while True:
        temp, pressure, humid = readBME280()
        fbuff.fill(0)
        fbuff.fill(0)
        fbuff.text("BME280", 0, 0, 1)
        fbuff.text("tmp:", 0, 16, 1)
        fatntext("%.2f"%(temp), 32, 10)
        fbuff.text("prs:", 0, 36, 1)
        fatntext("%.2f"%(pressure), 32, 30)
        fbuff.text("hum:", 0, 54, 1)
        fatntext("%.2f"%(humid), 32, 50)
        oledshow()
        time.sleep(0.11)

if devname == "SHT31D":
    from SHT31D_funcs import SHT31Dsetup, readSHT31D
    SHT31Dsetup(i2c)
    while True:
        tmp, humid = readSHT31D()
        fbuff.fill(0)
        fbuff.text("SHT31-D", 0, 0, 1)
        fbuff.text("temperature:", 10, 10, 1)
        fatntext("%.2f"%tmp, 20, 22)
        fbuff.text("humidity:", 10, 40, 1)
        fatntext("%.2f"%humid, 20, 50)
        oledshow()
        time.sleep(0.1)
        
if devname == "MLX90621":
    fbuff.text("11", 50, 50, 1)
    oledshow()
    from MLX90621_funcs import measure, cellfillplot
    fbuff.text("22", 59, 50, 1)
    oledshow()
    while True:
        temperatures = measure()
        tmin = min(temperatures)
        tmax = max(tmin+0.001, max(temperatures))
        fbuff.fill(0)
        fbuff.text("MLX90621", 0, 0, 1)
        for i in range(64):
            cellfillplot(fbuff, i//4, 1+(3-(i%4)), ((temperatures[i] - tmin)/(tmax - tmin))**2)
        fbuff.text("min=%.3f"%tmin, 12, 48)
        fbuff.text("max=%.3f"%tmax, 12, 56)
        oledshow()

if devname == "PX4PITOT":
    while True:
        k = i2c.readfrom(0x28, 4)
        status = k[0] & 0xC0; 
        fbuff.fill(0)
        fbuff.text("PX4PITOT", 0, 0, 1)
        if status != 0xC0:
            rawpress = ((k[0] & 0x3F) << 8) | (k[1]) 
            rawtemp = (k[2] << 3) | (k[3] >> 5)
            psi = (rawpress)*(1.0/(0x3FFF*0.4)) - 1.25
            diffpressure = psi*6894.75728
            temp = (rawtemp)*(200.0/0x7FF) - 50
            #airspeed = math.sqrt(abs(2*getdiffpressure())/1.2);
            fbuff.text("diffpress:", 10, 10, 1)
            fatntext("%.2f"%diffpressure, 20, 22)
            fbuff.text("temp:", 10, 40, 1)
            fatntext("%.2f"%temp, 20, 50)
        else:
            fbuff.text("BAD", 20, 20, 1)
        oledshow()
        time.sleep(0.2)
        
if devname == "BNO055":
    from BNO055serial_funcs import InitBNO055, BNO055absorientation, BNO055calibstat
    InitBNO055(uart)
    while True:
        fbuff.fill(0)
        calibstat = None
        try:
            pitch, roll, orient = BNO055absorientation()
            calibstat = BNO055calibstat()
        except Exception as e:
            fbuff.text("BNO055", 0, 0, 1)
            fbuff.text(e.args[0], 0, 16, 1)
        if calibstat is not None:  
            fbuff.text("BNO055 :{0:08b}".format(calibstat), 0, 0, 1)
            fbuff.text("pitch:", 8, 16, 1)
            fatntext("%d"%int(pitch), 64, 10)
            fbuff.text("roll:", 16, 32, 1)
            fatntext("%d"%int(roll), 64, 28)
            fbuff.text("orient:", 0, 52, 1)
            fatntext("%d"%int(orient), 64, 48)
        oledshow()
        time.sleep(0.1)
        
if devname == "GPS":
    while True:
        x = uart.readline()
        if x is not None:
            fbuff.fill(0)
            fbuff.text("GPS serial", 0, 0, 1)
            x = x.strip()
            for i in range(len(x)//16 + 1):
                fbuff.text(x[i*16:i*16+16], 0, 15+i*8, 1)
            oledshow()
            time.sleep_ms(100)
        time.sleep_ms(50)


# read any GPS serial signals on 
fbuff.text("Z", 120, 56, 1)
oledshow()


Sent 220 lines (6885 bytes) to main.py.


In [42]:
%rebootdevice

normal repl mode not detected b'\r\nMicroPython v1.9.3-603-gfb7dabb9 on 2018-05-02; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> \r\n>>> \r\nPYB: soft reboot\r\n'
not command mode
[Timed out waiting for recognizable response]


In [10]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [1070]:
import dmain

I (24502) uart: ALREADY NULL
..........................

*** Sending Ctrl-C



Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
  File "dmain.py", line 194, in <module>
KeyboardInterrupt: 


In [1056]:
import os
os.rename("main.py", "dmain.py")

In [876]:
help(machine)

object <module 'umachine'> is of type module
  __name__ -- umachine
  mem8 -- <8-bit memory>
  mem16 -- <16-bit memory>
  mem32 -- <32-bit memory>
  freq -- <function>
  reset -- <function>
  unique_id -- <function>
  sleep -- <function>
  deepsleep -- <function>
  idle -- <function>
  disable_irq -- <function>
  enable_irq -- <function>
  time_pulse_us -- <function>
  Timer -- <class 'Timer'>
  WDT -- <class 'WDT'>
  SLEEP -- 2
  DEEPSLEEP -- 4
  Pin -- <class 'Pin'>
  Signal -- <class 'Signal'>
  TouchPad -- <class 'TouchPad'>
  ADC -- <class 'ADC'>
  DAC -- <class 'DAC'>
  I2C -- <class 'I2C'>
  PWM -- <class 'PWM'>
  RTC -- <class 'RTC'>
  SPI -- <class 'SoftSPI'>
  UART -- <class 'UART'>
  reset_cause -- <function>
  HARD_RESET -- 2
  PWRON_RESET -- 1
  WDT_RESET -- 3
  DEEPSLEEP_RESET -- 4
  SOFT_RESET -- 5
  wake_reason -- <function>
  PIN_WAKE -- 1
  EXT0_WAKE -- 1
  EXT1_WAKE -- 2
  TIMER_WAKE -- 3
  TOUCHPAD_WAKE -- 4
  ULP_WAKE -- 5


In [1107]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [11]:
import machine
x = machine.ADC(machine.Pin(34))
help(x)

object ADC(Pin(34)) is of type ADC
  read -- <function>
  atten -- <function>
  width -- <classmethod>
  ATTN_0DB -- 0
  ATTN_2_5DB -- 1
  ATTN_6DB -- 2
  ATTN_11DB -- 3
  WIDTH_9BIT -- 0
  WIDTH_10BIT -- 1
  WIDTH_11BIT -- 2
  WIDTH_12BIT -- 3


In [12]:
x.atten(x.ATTN_11DB)
x.width(x.WIDTH_12BIT)
print(x.read())

370


In [1121]:
print(time.ticks_ms())

685210


In [13]:
import time
for i in range(20):
    x.atten(x.ATTN_11DB)
    print(x.read())
    time.sleep(0.5)

2679
4095
4095
4095
4095
4095
4095
2791
4095
4095
3071
2637
4095
2555
2612
2631
2609
2630
2631
2633


In [30]:
%serialconnect

Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [28]:
%disconnect

serial exception on close write failed: [Errno 5] Input/output error

Closing serial Serial<id=0x7f0e70925668, open=True>(port='/dev/ttyUSB0', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)


In [31]:
#%sendtofile main.py

from OLED_driver import i2c, fbuff, oledshow, doublepixels, fatntext
import time, machine

x = machine.ADC(machine.Pin(36))
while True:
    fbuff.fill(0)
    fbuff.text("Pin36 ADC", 56, 56, 1)
    x.atten(x.ATTN_0DB)
    fatntext(str(x.read()), 0, 0)
    x.atten(x.ATTN_6DB)
    fatntext(str(x.read()), 0, 20)
    x.atten(x.ATTN_11DB)
    fatntext(str(x.read()), 0, 40)
    oledshow()
    time.sleep(0.1)

...........................................................

*** Sending Ctrl-C



Traceback (most recent call last):
  File "<stdin>", line 17, in <module>
KeyboardInterrupt: 


In [15]:
%rebootdevice

repl is in normal command mode
[\r\x03\x03] b'\r\nMicroPython v1.9.2-445-g84035f0f on 2017-11-29; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> \r\n>>> \r\nPYB: soft reboot\r\nTraceback (most recent call last):\r\n  File "main.py", line 1, in <module>\r\nImportError: no module named \'OLED_driver\'\r\nMicroPython v1.9.2-445-g84035f0f on 2017-11-29; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> \r\n>>> \r\n>>> '
[\r\x01] b'\r\n>>> \r\nraw REPL; CTRL-B to exit\r\n>'